In [39]:
import requests
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import collections  

In [13]:
url = f"https://api.opendota.com/api/teams"
response = requests.get(url)
teams_data = response.json()

In [14]:
falcons_id = []
for teams in teams_data: 
    if teams['name'] == 'Team Falcons': 
        falcons_id = teams['team_id']
        print(falcons_id)

9247354


In [17]:
def team_info(falcons_id):
    url = f"https://api.opendota.com/api/teams/{falcons_id}"
    response = requests.get(url)
    return response.json()

In [19]:
def win_rate(falcons_id):
    team_data = team_info(falcons_id)
    wins = team_data['wins']
    losses = team_data['losses']
    total = wins + losses
    win_rate = (wins / total) * 100 if total > 0 else 0
    return round(win_rate, 1)
print(f"A Falcons tem {win_rate(falcons_id)}% de WinRate")

A Falcons tem 73.7% de WinRate


In [46]:
def get_matches(falcons_id):
    url = f"https://api.opendota.com/api/teams/{falcons_id}/matches"
    response = requests.get(url)
    matches = response.json()
    return matches[:4]

In [58]:
matches = get_matches(falcons_id)

In [47]:
def format_duration(duration):
    hours = duration // 3600             
    minutes = (duration % 3600) // 60    
    seconds = duration % 60    
    if hours > 0:
        return f"{hours}h {minutes}m {seconds}s"
    else:
        return f"{minutes}m {seconds}s"

In [48]:
def team_side(match):
    if match['radiant']:
        return True  
    else:
        return False

In [49]:
for match in matches:
    is_radiant = team_side(match)
    if (is_radiant and match['radiant_win']) or (not is_radiant and not match['radiant_win']):
        resultado = 'Vitória'
    else:
        resultado = 'Derrota'
    duration_formatted = format_duration(match['duration'])
    print(f"Resultado: {resultado}, Duração: {duration_formatted}")

Resultado: Derrota, Duração: 41m 12s
Resultado: Vitória, Duração: 42m 27s
Resultado: Vitória, Duração: 33m 46s
Resultado: Vitória, Duração: 40m 33s


In [50]:
match_ids = []
for match in matches:
    match_ids.append(match['match_id']) 

In [51]:
count = 0
count1 = 1
total_kils = 0

for match in matches:
    is_radiant = team_side(match)
    radiant_score = match['radiant_score']
    dire_score = match['dire_score']
    total_score_match = radiant_score + dire_score
    total_kils += total_score_match 
    print(f"A partida {count1} teve {total_score_match} abates")
    if is_radiant == True:
        print(f"A Falcons teve {radiant_score} kills")
        print(f"O outro time teve {dire_score} kills")
    
    elif is_radiant == False:
        print(f"A Falcons teve {dire_score} kills")
        print(f"O outro time teve {radiant_score} kills")

    count += 1
    count1 += 1
    
media = (total_kils/count)
print(f"Media de kils nas ultimas {count} partidas foi de {media} kils")
print(f"Total de kills em todas as partidas: {total_kils}")

A partida 1 teve 50 abates
A Falcons teve 12 kills
O outro time teve 38 kills
A partida 2 teve 35 abates
A Falcons teve 28 kills
O outro time teve 7 kills
A partida 3 teve 36 abates
A Falcons teve 26 kills
O outro time teve 10 kills
A partida 4 teve 48 abates
A Falcons teve 29 kills
O outro time teve 19 kills
Media de kils nas ultimas 4 partidas foi de 42.25 kils
Total de kills em todas as partidas: 169


In [52]:
total_duration = 0
for match in matches:
    total_duration += match['duration']  
media = total_duration / 4
print(f"O tempo total de duracao das partidas foi de {format_duration(total_duration)}")  
print(f"A media de tempo de cada partida foi de {format_duration(media)}")  

O tempo total de duracao das partidas foi de 2h 37m 58s
A media de tempo de cada partida foi de 39.0m 29.5s


In [53]:
def get_match_detail(match_ids):
    url = f"https://api.opendota.com/api/matches/{match_id}"
    response = requests.get(url)
    return response.json()

In [54]:
url = f'https://api.opendota.com/api/teams/{falcons_id}/players'
response = requests.get(url) 
players1 = response.json()
players_name = []
for player in players1:
    account_id = player.get('name')
    in_team = player.get("is_current_team_member")
    if account_id and in_team == True:
        players_name.append(account_id)

In [55]:
mean_kda = {player_name: [] for player_name in players_name} 
mean_kills = {player_name: [] for player_name in players_name} 
mean_deaths = {player_name: [] for player_name in players_name} 
mean_assists = {player_name: [] for player_name in players_name} 

for match_id in match_ids:
    match_data = get_match_detail(match_id)
    players = match_data['players']
    
    for player in players:
        jogador_nome = player.get('name', 'Desconhecido')
        if jogador_nome in players_name:
            kills = player['kills']
            deaths = player['deaths']
            assists = player['assists']
            kda = (kills + assists) / (deaths if deaths > 0 else 1) 

            mean_kda[jogador_nome].append(kda)
            mean_kills[jogador_nome].append(kills)
            mean_deaths[jogador_nome].append(deaths)
            mean_assists[jogador_nome].append(assists)


for jogador_nome in players_name:
    kda_lista = mean_kda[jogador_nome]
    kills_lista = mean_kills[jogador_nome]
    deaths_lista = mean_deaths[jogador_nome]
    assists_lista = mean_assists[jogador_nome]
    
    kda_medio = sum(kda_lista) / len(kda_lista) if kda_lista else 0
    kills_medio = sum(kills_lista) / len(kills_lista) if kills_lista else 0
    deaths_medio = sum(deaths_lista) / len(deaths_lista) if deaths_lista else 0
    assists_medio = sum(assists_lista) / len(assists_lista) if assists_lista else 0
    
    if kda_medio > 0:    
        print(f"O jogador {jogador_nome} tem uma média de {kda_medio:.2f} de KDA.")
        print(f" - Média de kills: {kills_medio:.2f}")
        print(f" - Média de mortes: {deaths_medio:.2f}")
        print(f" - Média de assistências: {assists_medio:.2f}")

O jogador AMMAR_THE_F tem uma média de 5.06 de KDA.
 - Média de kills: 4.50
 - Média de mortes: 4.00
 - Média de assistências: 12.00
O jogador Cr1t- tem uma média de 7.36 de KDA.
 - Média de kills: 3.00
 - Média de mortes: 5.50
 - Média de assistências: 14.75
O jogador skiter tem uma média de 13.50 de KDA.
 - Média de kills: 5.00
 - Média de mortes: 2.75
 - Média de assistências: 10.25
O jogador Sneyking tem uma média de 6.35 de KDA.
 - Média de kills: 3.50
 - Média de mortes: 4.00
 - Média de assistências: 10.25
O jogador Malr1ne tem uma média de 13.34 de KDA.
 - Média de kills: 7.75
 - Média de mortes: 2.25
 - Média de assistências: 10.75


In [56]:
def get_hero_mapping():
    url = "https://api.opendota.com/api/heroes"
    response = requests.get(url)
    heroes_data = response.json()
    
    hero_mapping = {hero['id']: hero['localized_name'] for hero in heroes_data}
    return hero_mapping

hero_mapping = get_hero_mapping()

In [57]:
hero_counter = collections.Counter()

for match_id in match_ids:
    match_data = get_match_detail(match_id)
    players = match_data['players']
    
    for player in players:
        hero_id = player['hero_id']
        hero_counter[hero_id] += 1

top_heroes = hero_counter.most_common(5)

print("Heróis que mais apareceram nas últimas partidas:")
for hero_id, count in top_heroes:
    hero_name = hero_mapping.get(hero_id, "Desconhecido")
    print(f"Herói {hero_name} apareceu {count} vezes.")

Heróis que mais apareceram nas últimas partidas:
Herói Tusk apareceu 3 vezes.
Herói Pugna apareceu 2 vezes.
Herói Dawnbreaker apareceu 2 vezes.
Herói Mars apareceu 2 vezes.
Herói Phoenix apareceu 2 vezes.
